In [3]:
import os
import pandas as pd
import glob

# 💡 이 설정값들을 사용자의 환경에 맞게 반드시 수정하세요!
# -------------------------------------------------------------
BASE_DOWNLOAD_DIR = r'F:\ML\dataset'      # 👈 장르별 CSV 파일들이 저장된 기본 경로
CSV_FILENAME_PREFIX = 'screenshot_tag_vectors_' # 👈 장르별 CSV 파일명의 공통 시작 부분
FINAL_CSV_FILENAME = 'final_unique_tag_vectors.csv' # 👈 최종 통합 파일명
# -------------------------------------------------------------

def merge_csv_files():
    """
    지정된 경로의 모든 장르별 CSV 파일을 통합하고 파일명 중복을 제거하여 저장합니다.
    """
    search_path = os.path.join(BASE_DOWNLOAD_DIR, f'{CSV_FILENAME_PREFIX}*.csv')
    
    # 1. 파일명 패턴에 맞는 모든 CSV 파일 목록 가져오기
    csv_files = glob.glob(search_path)
    
    if not csv_files:
        print(f"❌ 오류: 경로 {BASE_DOWNLOAD_DIR}에서 '{CSV_FILENAME_PREFIX}*.csv' 패턴에 맞는 CSV 파일을 찾을 수 없습니다.")
        return

    print(f"🔍 총 {len(csv_files)}개의 CSV 파일을 발견했습니다. 통합을 시작합니다...")
    
    all_data_frames = []
    
    # 2. 모든 CSV 파일을 읽어 DataFrame 리스트에 저장
    for f in csv_files:
        try:
            df = pd.read_csv(f)
            all_data_frames.append(df)
            print(f"   -> 파일 로드 완료: {os.path.basename(f)} ({len(df)}행)")
        except Exception as e:
            print(f"   -> ⚠️ 파일 로드 실패 ({os.path.basename(f)}): {e}")

    if not all_data_frames:
        print("❌ 오류: 성공적으로 로드된 CSV 파일이 없습니다.")
        return

    # 3. 모든 DataFrame을 하나로 통합
    df_combined = pd.concat(all_data_frames, ignore_index=True)
    total_rows_before = len(df_combined)
    print("-" * 50)
    print(f"총 통합된 행 수 (중복 포함): {total_rows_before}행")

    # 4. 'filename' 열을 기준으로 중복 제거 (첫 번째 행 유지)
    # 한 스크린샷이 여러 장르에 매칭되어 CSV에 중복 기록되었을 경우,
    # 중복된 파일명은 하나만 남기고 나머지는 제거합니다.
    df_unique = df_combined.drop_duplicates(subset=['filename'], keep='first')
    total_rows_after = len(df_unique)
    
    duplicates_removed = total_rows_before - total_rows_after
    
    # 5. 최종 파일 저장
    final_output_path = os.path.join(BASE_DOWNLOAD_DIR, FINAL_CSV_FILENAME)
    df_unique.to_csv(final_output_path, index=False, encoding='utf-8')

    print(f"✅ 중복 제거 완료. {duplicates_removed}개의 중복된 스크린샷이 제거되었습니다.")
    print("-" * 50)
    print(f"최종 통합된 고유 스크린샷 수: {total_rows_after}행")
    print(f"최종 파일 저장 경로: {final_output_path}")
    print("-" * 50)

# 함수 실행
merge_csv_files()

🔍 총 11개의 CSV 파일을 발견했습니다. 통합을 시작합니다...
   -> 파일 로드 완료: screenshot_tag_vectors_Action.csv (315200행)
   -> 파일 로드 완료: screenshot_tag_vectors_Adventure.csv (291501행)
   -> 파일 로드 완료: screenshot_tag_vectors_Puzzle.csv (113452행)
   -> 파일 로드 완료: screenshot_tag_vectors_Racing.csv (29494행)
   -> 파일 로드 완료: screenshot_tag_vectors_RPG.csv (136614행)
   -> 파일 로드 완료: screenshot_tag_vectors_Sandbox.csv (47764행)
   -> 파일 로드 완료: screenshot_tag_vectors_Shooter.csv (78453행)
   -> 파일 로드 완료: screenshot_tag_vectors_Simulation.csv (155824행)
   -> 파일 로드 완료: screenshot_tag_vectors_Sports.csv (34998행)
   -> 파일 로드 완료: screenshot_tag_vectors_Strategy.csv (148014행)
   -> 파일 로드 완료: screenshot_tag_vectors_Survival.csv (60314행)
--------------------------------------------------
총 통합된 행 수 (중복 포함): 1411628행
✅ 중복 제거 완료. 781602개의 중복된 스크린샷이 제거되었습니다.
--------------------------------------------------
최종 통합된 고유 스크린샷 수: 630026행
최종 파일 저장 경로: F:\ML\dataset\final_unique_tag_vectors.csv
---------------------------------------------

In [5]:
import os
import pandas as pd
import glob
from typing import Set

# 💡 이 설정값들을 환경에 맞게 반드시 수정하세요!
# -------------------------------------------------------------
BASE_DIR = r'F:\ML\dataset'                 # 👈 모든 데이터 폴더의 상위 경로
FINAL_CSV_FILENAME = 'final_unique_tag_vectors.csv' # 👈 통합된 원본 CSV 파일명
VALIDATION_DIR_NAME = 'Validation_Set' # 👈 검증 이미지가 저장된 폴더명
VALIDATION_CSV_FILENAME = 'validation_tag_vectors.csv' # 👈 최종 저장될 검증용 CSV 파일명
# -------------------------------------------------------------


def create_validation_csv():
    """
    Validation_Set 폴더 내의 이미지 파일명만 필터링하여 검증용 CSV를 생성합니다.
    """
    
    validation_base_dir = os.path.join(BASE_DIR, VALIDATION_DIR_NAME)
    source_csv_path = os.path.join(BASE_DIR, FINAL_CSV_FILENAME)
    output_csv_path = os.path.join(BASE_DIR, VALIDATION_CSV_FILENAME)
    
    # 1. Validation_Set 폴더 내 모든 파일명(filename) 확보
    validation_filenames: Set[str] = set()
    
    print(f"🔍 {VALIDATION_DIR_NAME} 폴더에서 파일명 목록을 수집 중...")
    
    # Validation_Set 아래의 모든 장르 폴더를 순회
    # glob.glob 재귀적 검색 (**/*)을 사용하여 모든 하위 폴더의 파일을 찾습니다.
    # [pj]g는 .jpg와 .png 파일을 의미
    search_pattern = os.path.join(validation_base_dir, '**', '*.[pj][np]g')
    
    # recursive=True를 사용하여 하위 폴더까지 탐색
    all_validation_files = glob.glob(search_pattern, recursive=True)

    for file_path in all_validation_files:
        validation_filenames.add(os.path.basename(file_path))

    if not validation_filenames:
        print(f"❌ 오류: {VALIDATION_DIR_NAME} 폴더에서 이미지 파일이 발견되지 않았습니다. 경로를 확인하세요.")
        return

    print(f"✅ {VALIDATION_DIR_NAME}에서 {len(validation_filenames)}개의 고유 파일명을 확보했습니다.")
    
    # 2. 원본 통합 CSV 파일 로드
    try:
        df_full = pd.read_csv(source_csv_path)
        print(f"📊 원본 CSV 파일 로드 완료. 총 {len(df_full)}행.")
    except FileNotFoundError:
        print(f"❌ 오류: 원본 CSV 파일을 찾을 수 없습니다. 경로를 확인하세요: {source_csv_path}")
        return
    except Exception as e:
        print(f"❌ 오류: CSV 파일 로드 중 문제가 발생했습니다: {e}")
        return

    # 3. 'filename' 열을 기준으로 필터링
    # df_full['filename'].isin(validation_filenames)은 filename이 set에 있는지 True/False로 반환
    df_validation = df_full[df_full['filename'].isin(validation_filenames)]

    # 4. 결과 출력 및 저장
    if len(df_validation) != len(validation_filenames):
        print(f"⚠️ 경고: CSV에서 {len(df_validation)}개, 폴더에서 {len(validation_filenames)}개 일치. 불일치하는 파일이 있을 수 있습니다.")

    df_validation.to_csv(output_csv_path, index=False, encoding='utf-8')

    print("-" * 50)
    print(f"🎉 검증용 CSV 생성 완료. 총 {len(df_validation)}행 저장.")
    print(f"저장 경로: {output_csv_path}")
    print("-" * 50)


# 함수 실행
create_validation_csv()

🔍 Validation_Set 폴더에서 파일명 목록을 수집 중...
✅ Validation_Set에서 33000개의 고유 파일명을 확보했습니다.
📊 원본 CSV 파일 로드 완료. 총 630026행.
--------------------------------------------------
🎉 검증용 CSV 생성 완료. 총 33000행 저장.
저장 경로: F:\ML\dataset\validation_tag_vectors.csv
--------------------------------------------------


In [ ]:
import os
import random
import shutil
import glob
from typing import List, Dict, Set

# 💡 설정값들을 환경에 맞게 반드시 수정하세요!
# -------------------------------------------------------------
BASE_DIR = r'F:\ML\dataset'                 # 👈 모든 데이터 폴더의 상위 경로
SOURCE_DIR_NAME = 'Processed_224x224'  # 👈 전처리된 이미지가 저장된 폴더명
VALIDATION_DIR_NAME = 'Validation_Set' # 👈 검증 데이터셋을 저장할 새 폴더명
FILES_TO_MOVE_PER_GENRE = 2000         # 👈 장르별 목표 파일 수
# -------------------------------------------------------------

TARGET_TAGS: List[str] = [
    "Adventure", "Action", "RPG", "Strategy", "Simulation", "Sports", "Racing", 
    "Puzzle", "Sandbox", "Shooter", "Survival", "Fighting", "Music"
]

def get_file_count_and_sort_tags(source_base_dir: str) -> List[str]:
    """
    각 장르 폴더의 파일 수를 세어 데이터가 적은 장르부터 오름차순으로 정렬합니다.
    """
    tag_counts: Dict[str, int] = {}
    print("🔍 각 장르별 파일 수를 확인 중...")
    
    for tag in TARGET_TAGS:
        source_genre_dir = os.path.join(source_base_dir, tag)
        if os.path.isdir(source_genre_dir):
            # 이미지 파일 (jpg, png)만 카운트
            file_paths = glob.glob(os.path.join(source_genre_dir, '*.[pj][np]g'))
            tag_counts[tag] = len(file_paths)
        else:
            tag_counts[tag] = 0
            
    # 파일 수 오름차순으로 태그 정렬
    sorted_tags = sorted(tag_counts.keys(), key=lambda t: tag_counts[t])
    
    print("✅ 데이터가 적은 장르부터 순서대로 처리합니다:")
    for tag in sorted_tags:
        print(f"   - {tag}: {tag_counts[tag]}개")
        
    return sorted_tags

# 💡 새로운 헬퍼 함수: 학습 데이터셋에서 검증 파일과 중복되는 모든 파일 삭제
def remove_duplicates_from_all_train_dirs(
    base_dir: str, 
    source_dir_name: str, 
    filename_to_remove: str, 
    target_tags: List[str]
):
    """
    모든 장르 폴더를 순회하며 지정된 파일명과 동일한 파일을 삭제합니다.
    """
    source_base_dir = os.path.join(base_dir, source_dir_name)

    for tag in target_tags:
        train_genre_dir = os.path.join(source_base_dir, tag)
        file_path = os.path.join(train_genre_dir, filename_to_remove)
        
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
                # print(f"    [중복 제거] {tag} 폴더에서 {filename_to_remove} 삭제 완료.")
            except Exception as e:
                print(f"    [중복 제거] ❌ 삭제 오류 ({file_path}): {e}")

def create_unique_validation_set():
    """
    각 장르에서 2000개의 고유한 파일을 랜덤 선택하여 검증 데이터셋으로 이동시킵니다.
    모든 검증 폴더를 통틀어 파일명이 중복되지 않도록 보장합니다.
    """
    
    source_base_dir = os.path.join(BASE_DIR, SOURCE_DIR_NAME)
    validation_base_dir = os.path.join(BASE_DIR, VALIDATION_DIR_NAME)
    os.makedirs(validation_base_dir, exist_ok=True)
    
    # 데이터가 적은 장르부터 처리하도록 태그 순서 정렬
    sorted_tags = get_file_count_and_sort_tags(source_base_dir)

    print("-" * 50)
    print(f"검증 데이터셋 분리 시작. 장르별 목표: {FILES_TO_MOVE_PER_GENRE}개")
    print("-" * 50)
    
    # 1. 전역 추적: 이미 검증 데이터셋에 포함된 모든 파일명 (중복 방지용)
    global_validation_filenames: Set[str] = set()
    
    # 기존에 이동된 파일이 있다면 목록에 추가하여 재실행 시 중복되지 않도록 함
    for tag in TARGET_TAGS:
        validation_genre_dir = os.path.join(validation_base_dir, tag)
        if os.path.isdir(validation_genre_dir):
            moved_files = glob.glob(os.path.join(validation_genre_dir, '*.[pj][np]g'))
            global_validation_filenames.update({os.path.basename(f) for f in moved_files})

    total_moved = len(global_validation_filenames)
    print(f"🚨 현재 검증 데이터셋에 이미 {total_moved}개의 고유한 파일이 있습니다. 작업을 이어갑니다.")

    # 2. 정렬된 순서대로 장르별 파일 이동 시작
    for tag in sorted_tags:
        source_genre_dir = os.path.join(source_base_dir, tag)
        validation_genre_dir = os.path.join(validation_base_dir, tag)
        os.makedirs(validation_genre_dir, exist_ok=True)
        
        # 3. 현재 장르 폴더에 이미 이동된 파일 수 확인
        current_moved_in_genre = len(glob.glob(os.path.join(validation_genre_dir, '*.[pj][np]g')))
        
        if current_moved_in_genre >= FILES_TO_MOVE_PER_GENRE:
            print(f"[{tag}] ✅ 목표 수 ({FILES_TO_MOVE_PER_GENRE}개) 달성 완료. 스킵합니다.")
            continue

        needed_to_move = FILES_TO_MOVE_PER_GENRE - current_moved_in_genre
        
        # 4. 원본 폴더에서 이동 가능한 모든 파일 목록 가져오기
        all_source_files = glob.glob(os.path.join(source_genre_dir, '*.[pj][np]g'))
        
        # 5. 전역 중복을 확인하여 이동 후보 파일 목록 생성
        # (아직 검증 데이터셋의 어떤 폴더에도 들어가지 않은 파일만 후보가 됨)
        unique_candidates = [
            f for f in all_source_files 
            if os.path.basename(f) not in global_validation_filenames
        ]

        # 6. 랜덤 선택 및 이동
        if not unique_candidates:
            print(f"[{tag}] ⚠️ 이동 가능한 고유 파일이 부족하거나 남아있지 않습니다. (현재: {current_moved_in_genre}개)")
            continue
            
        if len(unique_candidates) < needed_to_move:
            # 원본에 남아있는 고유 파일이 부족한 경우
            selected_files = unique_candidates
            print(f"[{tag}] ⚠️ 파일 부족: {len(selected_files)}개만 이동합니다.")
        else:
            # 필요한 파일 수만큼 랜덤 선택
            selected_files = random.sample(unique_candidates, needed_to_move)

        moved_count = 0
        
        for source_path in selected_files:
            filename = os.path.basename(source_path)
            destination_path = os.path.join(validation_genre_dir, filename)
            
            try:
                # 파일을 원본 폴더에서 검증 폴더로 이동 (원본 폴더에서는 삭제됨)
                shutil.move(source_path, destination_path)
                
               # 2. 💡 핵심 추가 로직: 나머지 모든 장르 폴더에서 동일 파일명을 삭제
                remove_duplicates_from_all_train_dirs(
                    BASE_DIR, 
                    SOURCE_DIR_NAME, 
                    filename, 
                    TARGET_TAGS
                )
                
                # 3. 전역 추적 셋에 파일명 추가 및 카운트
                global_validation_filenames.add(filename)
                moved_count += 1
            except Exception as e:
                print(f"[{tag}] ❌ 파일 이동 오류 ({filename}): {e}")

        print(f"[{tag}] 이동 완료. 총 {current_moved_in_genre + moved_count}개 확보 (이번에 {moved_count}개 이동).")

    print("-" * 50)
    print(f"🎉 모든 장르 분리 완료.")
    print(f"최종적으로 검증 데이터셋에 확보된 고유 이미지 수: {len(global_validation_filenames)}장")
    print(f"검증 데이터셋 경로: {validation_base_dir}")
    print("-" * 50)

# 함수 실행
create_unique_validation_set()